In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet transformers
!pip install --quiet torch

In [3]:
!pip install flask_ngrok 


In [4]:
file_path = '/content/drive/MyDrive/LLM/wiki.txt'

# NEW CODE 03/04/2023

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from flask import Flask, request, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from flask import Flask, request, jsonify

# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Define a function to encode the input question and context as input tensors for the model
def encode(question, context):
    # Tokenize the question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')

    # Pad the input sequences to a fixed length of 128 tokens
    max_length = 128
    padded_inputs = {
        key: torch.nn.functional.pad(input_tensor, (0, max_length - input_tensor.shape[-1]), mode='constant', value=0)
        for key, input_tensor in inputs.items()
    }

    # Return the padded inputs
    return padded_inputs

# Define a function to perform question answering on a given context
def qa(question, context):
    inputs = encode(question, context)
    input_ids = inputs["input_ids"].tolist()[0]
    
    outputs = bert_model(**inputs)
    start_scores = torch.tensor(outputs.start_logits)
    end_scores = torch.tensor(outputs.end_logits)
    
    # Get the indices of the best answer
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1
    
    # Convert the token ids back to text
    answer_tokens = input_ids[answer_start:answer_end]
    answer = tokenizer.decode(answer_tokens)

    # Return the answer as a JSON response
    return answer



# Define an API endpoint for the question answering model
#@app.route('/qa', methods=['POST'])
def qa_endpoint(data):
    # Parse the input data from the request
    #data = request.get_json()
    question = data['question']
    file_path = data['file_path']
    
    # Read the text file
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Split the text into chunks of 512 tokens or less
    chunk_size = 512
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    
    # Process each chunk and concatenate the answers
    answers = []
    for chunk in chunks:
        answer = qa(question, chunk)
        answers.append(answer)
    
    final_answer = ' '.join(answers)
    
    # Return the final answer as a JSON response
    print(final_answer)



with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

question = "What is the full form of NATEF?"
#question = "What is NATEF responsible for?"

#question = "under whom Apprentice technicians work ?"

context = text #"BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained transformer-based natural language processing model developed by Google."



# Define the input data as a dictionary
data = {'question': question, 'file_path': file_path}

qa_endpoint(data)


<ipython-input-7-3ca1044b727a>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_scores = torch.tensor(outputs.start_logits)
<ipython-input-7-3ca1044b727a>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_scores = torch.tensor(outputs.end_logits)


automotive technician apprentice technicians  national automotive technicians education foundation natef? [SEP] website national institute for automotive service excellence  universal technical institute ironment section 609 of the clear air act lisions fleet brication certified for select systems natef? [SEP] natef? [SEP] n inside a brick and mortar facility technological financial    tra natef? [SEP] desperson natef


# RUNNING CODE - 03/04/2023

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from flask import Flask, request, jsonify


# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Initialize a Flask app
#app = Flask(__name__)

# Define a function to encode the input question and context as input tensors for the model
def encode(question, context):
    # Tokenize the question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')

    # Pad the input sequences to a fixed length of 128 tokens
    max_length = 128
    padded_inputs = {
        key: torch.nn.functional.pad(input_tensor, (0, max_length - input_tensor.shape[-1]), mode='constant', value=0)
        for key, input_tensor in inputs.items()
    }

    # Return the padded inputs
    return padded_inputs

# Define an API endpoint for the question answering model
#@app.route('/qa', methods=['POST'])
def qa(data):
    # Parse the input data from the request
    #data = request.get_json()
    question = data['question']
    context = data['context']

    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    
    outputs = bert_model(**inputs)
    start_scores = torch.tensor(outputs.start_logits)
    end_scores = torch.tensor(outputs.end_logits)
    
    # Get the indices of the best answer
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1
    
    # Convert the token ids back to text
    answer_tokens = input_ids[answer_start:answer_end]
    answer = tokenizer.decode(answer_tokens)

    # Return the answer as a JSON response
    print("Answer: ", answer)



with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()
    text = "The National Automotive Technicians Education Foundation (NATEF) is responsible for evaluating technician training programs against standards developed by the automotive industry. NATEF accredits programs in four different categories: automotive, collision, trucks (diesel technology) and alternative fuels. Diesel mechanics have developed into a trade somewhat distinctive from gasoline-engine mechanics."

question = "What is NATEF?"
question = "What is NATEF responsible for?"

question = "how many categories does NATEF accredits programs ?"

question = "List all categories in which  NATEF accredits programs ?"
question = "What is the full form of NATEF?"


context = text #"BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained transformer-based natural language processing model developed by Google."

# Define the API endpoint URL
#url = 'http://localhost:5000/qa'

# Define the input data as a dictionary
data = {'question': question, 'context': context}

qa(data)


# Run the Flask app
#if __name__ == '__main__':
#    app.run(debug=True)


Answer:  the national automotive technicians education foundation


<ipython-input-10-b2b722f41c34>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_scores = torch.tensor(outputs.start_logits)
<ipython-input-10-b2b722f41c34>:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_scores = torch.tensor(outputs.end_logits)


# OLD CODE 


In [ ]:
file_path = '/content/drive/MyDrive/LLM/wiki.txt'

In [ ]:
file_path = '/content/drive/MyDrive/LLM/wiki.txt'

with open(file_path, 'r') as file:
    text = file.read()

import re
text = re.sub('\n', ' ', text) # remove line breaks

print (text)

import os
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import numpy as np


An auto mechanic (automotive technician in most of North America, light vehicle technician in British English, and motor mechanic in Australian English) is a mechanic who services and repairs automobiles, sometimes specializing in one or more automobile brands or sometimes working with any brand. In fixing cars, their main role is to diagnose and repair the problem accurately and quickly. They often have to quote prices for their customers before commencing work or after partial disassembly for inspection. Their job may involve the repair of a specific part or the replacement of one or more parts as assemblies. Basic vehicle maintenance is a fundamental part of a mechanic's work in modern industrialized countries, while in others they are only consulted when a vehicle is already showing signs of malfunction. Apprentice technicians work under master technicians for a specified number of years before they work on their own. Some areas have formal apprenticeship programs,  however many au

In [ ]:
# Instantiate the BERT tokenizer and encode the text
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
max_length = tokenizer.max_len_single_sentence
encoding = tokenizer.encode_plus(text, max_length=max_length, return_tensors="pt", truncation=True)

# Find answers in the text using the BERT model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Split the input text into smaller chunks and run the model separately on each chunk
chunk_size = 512
inputs = []
for i in range(0, encoding["input_ids"].size(1), chunk_size):
    input_ids = encoding["input_ids"][:, i:i+chunk_size]
    attention_mask = encoding["attention_mask"][:, i:i+chunk_size]
    inputs.append((input_ids, attention_mask))

outputs = []
for input_ids, attention_mask in inputs:
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    outputs.append(output)

# Concatenate the outputs from each chunk
start_scores = torch.cat([o.start_logits for o in outputs], dim=1)
end_scores = torch.cat([o.end_logits for o in outputs], dim=1)

# Find the tokens with the highest start and end scores
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze().tolist())
answer_tokens = tokens[start_index:end_index+1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

# Fill the answers list with the answer found in the text
if answer.strip() == "":
    answer = "example"
answers = [answer]

# Add start and end positions to the encoding
start_positions = []
end_positions = []
for answer in answers:
    start_idx = text.find(answer)
    end_idx = start_idx + len(answer) - 1
    start_positions.append(start_idx)
    end_positions.append(end_idx)
encoding["start_positions"] = torch.tensor(start_positions)
encoding["end_positions"] = torch.tensor(end_positions)

# Instantiate the BERT model and fine-tune it on the text
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Train the model for 3 epochs with a batch size of 1
for epoch in range(1):
    model.train()
    optimizer.zero_grad()
    # Feed the encoding to the model
    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]
    start_positions = encoding["start_positions"]
    end_positions = encoding["end_positions"]
    outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                    start_positions=start_positions, end_positions=end_positions)
    # Compute the loss and backpropagate
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Save the fine-tuned model to disk
model_dir = "trained_model"
os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_dir)


In [ ]:

# Use the fine-tuned model to answer a question
question = "What is the example text about?"
context = text
inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
input_ids = inputs["input_ids"][0:511]
attention_mask = inputs["attention_mask"][0:511]

print((inputs))

outputs = model(input_ids=input_ids, attention_mask=attention_mask)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Find the tokens with the highest start and end scores
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
answer_tokens = tokens[start_index:end_index+1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print("Question: ", question)
print("Answer: ", answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (2163 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[ 101, 2054, 2003,  ..., 3001, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


RuntimeError: ignored

In [ ]:
question = " what does NATEF  for?"

In [ ]:

# Use the fine-tuned model to answer a question

context = text
inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
outputs = model(input_ids=input_ids, attention_mask=attention_mask)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Find the tokens with the highest start and end scores
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
answer_tokens = tokens[start_index:end_index+1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print("Question: ", question)
print("Answer: ", answer)

Question:   what does NATEF  for?
Answer:  mydrive
